<a href="https://colab.research.google.com/github/dbaca14/HelloWorld/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author:** Dr. Shahriar Hossain <br>
**Topic of the code:** Generative Adversarial Networks (GANs) <br>
**Video explaining this code:** https://youtu.be/KQM6nuNsyhA <br>
**My YT Channel:** https://www.youtube.com/@C4A <br>
**Web:** https://computing4all.com/

In [ ]:
import torch
import torch.nn as nn

In [ ]:
# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Number of data points
n = 1000

# Generate random data for the first column
first_column = torch.rand(n, 1).to(device)

# Create second and third columns based on the relationship
second_column = 2 * first_column
third_column = 2 * second_column

# Combine columns to form data
data = torch.cat([first_column, second_column, third_column], dim=1)

In [ ]:
# Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 50),
            nn.ReLU(),
            nn.Linear(50, 3)
        )

    def forward(self, x):
        return self.model(x)



In [ ]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 50),
            nn.ReLU(),
            nn.Linear(50, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Initialize the models and move them to the device
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss and optimizers
criterion = nn.BCELoss()
optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.001)
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=0.001)

In [ ]:
# Training the GAN
num_epochs = 5000
for epoch in range(num_epochs):
    # Train discriminator
    optimizer_d.zero_grad()

    real_data = data
    real_labels = torch.ones(n, 1).to(device)
    outputs = discriminator(real_data)
    d_loss_real = criterion(outputs, real_labels)

    # Generate fake data
    noise = torch.randn(n, 3).to(device)
    fake_data = generator(noise)
    fake_labels = torch.zeros(n, 1).to(device)
    outputs = discriminator(fake_data.detach())
    d_loss_fake = criterion(outputs, fake_labels)

    # Backprop and optimize
    d_loss = d_loss_real + d_loss_fake
    d_loss.backward()
    optimizer_d.step()

    # Train generator
    optimizer_g.zero_grad()
    outputs = discriminator(fake_data)
    g_loss = criterion(outputs, real_labels)
    g_loss.backward()
    optimizer_g.step()

    # Print losses
    if (epoch+1) % 1000 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}")

Epoch [1000/5000], d_loss: 1.3725, g_loss: 0.7287
Epoch [2000/5000], d_loss: 1.3857, g_loss: 0.6946
Epoch [3000/5000], d_loss: 1.3976, g_loss: 0.7422
Epoch [4000/5000], d_loss: 1.3752, g_loss: 0.7155
Epoch [5000/5000], d_loss: 1.3852, g_loss: 0.7068


In [ ]:
# After training, generate some synthetic data
with torch.no_grad():
    test_noise = torch.randn(n, 3).to(device)
    generated_data = generator(test_noise).cpu().numpy()

# Print the first 10 rows of generated data
print("Generated Data (First 10 rows):")
for i in range(10):
    print(generated_data[i])

# To validate if relationships hold:
print("\nValidation (For the first 10 rows):")
for i in range(10):
    print(f"First: {generated_data[i][0]:.4f}, Expected Second: {2*generated_data[i][0]:.4f}, Actual Second: {generated_data[i][1]:.4f}")
    print(f"Second: {generated_data[i][1]:.4f}, Expected Third: {2*generated_data[i][1]:.4f}, Actual Third: {generated_data[i][2]:.4f}\n")

Generated Data (First 10 rows):
[0.7998012 1.660021  3.272396 ]
[0.37646177 0.76541066 1.5088928 ]
[0.40255162 0.83344334 1.6348023 ]
[0.1653687  0.34263033 0.6740363 ]
[0.5665213 1.1526009 2.2760208]
[0.14940399 0.3152167  0.62877667]
[0.80187684 1.660417   3.2814188 ]
[0.1325277  0.27246606 0.5379602 ]
[0.2489168 0.5163379 1.0093414]
[0.7744387 1.6064112 3.1636903]

Validation (For the first 10 rows):
First: 0.7998, Expected Second: 1.5996, Actual Second: 1.6600
Second: 1.6600, Expected Third: 3.3200, Actual Third: 3.2724

First: 0.3765, Expected Second: 0.7529, Actual Second: 0.7654
Second: 0.7654, Expected Third: 1.5308, Actual Third: 1.5089

First: 0.4026, Expected Second: 0.8051, Actual Second: 0.8334
Second: 0.8334, Expected Third: 1.6669, Actual Third: 1.6348

First: 0.1654, Expected Second: 0.3307, Actual Second: 0.3426
Second: 0.3426, Expected Third: 0.6853, Actual Third: 0.6740

First: 0.5665, Expected Second: 1.1330, Actual Second: 1.1526
Second: 1.1526, Expected Third: 2.3